In [ ]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import matplotlib

In [ ]:
sc = SparkContext('yarn-client')
input = sc.textFile('/iris-multiclass.csv')
input.count()

In [ ]:
parsedData = input.map(lambda line: array([float(x) for x in line.split(',')[0:4]]))

from pyspark.mllib.feature import PCA 
from pyspark.mllib.linalg import Vectors
points = parsedData.map(lambda point : Vectors.dense(point[0:4]))
pcamod = PCA(2).fit(points)
transformed = pcamod.transform(points)

In [ ]:
clusters = KMeans.train(transformed, 3, maxIterations=10, runs=10, initializationMode="k-means||")

print(transformed.first())
clusters.centers

In [ ]:
points = clusters.predict(transformed)
tpoints = transformed.map(lambda point : (point.toArray()[0], point.toArray()[1], clusters.predict(point.toArray())))
cluster0 = tpoints.filter(lambda cluster : cluster[2] == 0).map(lambda change : (change[0], change[1]))
cluster1 = tpoints.filter(lambda cluster : cluster[2] == 1).map(lambda change : (change[0], change[1]))
cluster2 = tpoints.filter(lambda cluster : cluster[2] == 2).map(lambda change : (change[0], change[1]))
cluster2.first()

In [ ]:
import matplotlib.pyplot as plt
x, y = zip(*transformed.collect())

In [ ]:
%pylab inline
plt.scatter(*zip(*cluster0.collect()), color='c')
plt.scatter(*zip(*cluster1.collect()), color='y')
plt.scatter(*zip(*cluster2.collect()), color='g')
x_center, y_center = zip(*clusters.centers)
plt.scatter(x_center, y_center, marker='x', color='r')
plt.show()